<a href="https://colab.research.google.com/github/pcbzmani/SQL_Learning/blob/main/null_safe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=0433a180fd7f0ef4c51fdc6052eef55cebccfe0c34c7135d34f3a55d0bf0733f
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [6]:
# prompt: create a spark dataframe with some null and valid values

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.appName("NullAndNotNullExample").getOrCreate()

data = [("James Smith", 30, "Block1"),
        ("Michael Rose,", 40, "Block2"),
        ("Robert Williams", 50, None),
        ("Maria Anne", 60, "Block1"),
        ("Jen Brown", 70, "Block2")
       ]

schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("address", StringType(), True)
])

df = spark.createDataFrame(data=data, schema=schema)

# df.show(truncate=False)
df.createOrReplaceTempView('customer')

+---------------+---+-------+
|name           |age|address|
+---------------+---+-------+
|James Smith    |30 |Block1 |
|Michael Rose,  |40 |Block2 |
|Robert Williams|50 |NULL   |
|Maria Anne     |60 |Block1 |
|Jen Brown      |70 |Block2 |
+---------------+---+-------+



In [13]:
df = spark.sql(f"""
    select * from customer
    where address != 'Block1'
""")

In [14]:
df.show(truncate = False)

+-------------+---+-------+
|name         |age|address|
+-------------+---+-------+
|Michael Rose,|40 |Block2 |
|Jen Brown    |70 |Block2 |
+-------------+---+-------+



In [15]:
df1 = spark.sql(f"""
    select * from customer
    where address = 'Block1'
""")

In [16]:
df1.show(truncate = False)

+-----------+---+-------+
|name       |age|address|
+-----------+---+-------+
|James Smith|30 |Block1 |
|Maria Anne |60 |Block1 |
+-----------+---+-------+



In [19]:
df2 = spark.sql(f"""
    select * from customer
    where !(address <=> 'Block1')
""")

In [20]:
df2.show(truncate = False)

+---------------+---+-------+
|name           |age|address|
+---------------+---+-------+
|Michael Rose,  |40 |Block2 |
|Robert Williams|50 |NULL   |
|Jen Brown      |70 |Block2 |
+---------------+---+-------+



In [21]:
spark.sql(" Select 1 = 1, NULL = NULL, 2 = NULL").show(truncate=False)

+-------+-------------+----------+
|(1 = 1)|(NULL = NULL)|(2 = NULL)|
+-------+-------------+----------+
|true   |NULL         |NULL      |
+-------+-------------+----------+



In [22]:
spark.sql(" Select 1 <=> 1, NULL <=> NULL, 2 <=> NULL").show(truncate=False)

+---------+---------------+------------+
|(1 <=> 1)|(NULL <=> NULL)|(2 <=> NULL)|
+---------+---------------+------------+
|true     |true           |false       |
+---------+---------------+------------+

